# Проект: "Отток клиентов"

**Задача проекта**:

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.


В рамках проекта мы проведем следующие **этапы исследования**: 

1) Ознакомимся с данными и подготовим их для дальнейшего исследования;

2) Исследуем баланс классов, обучим модель без учета дисбаланса;

3) Улучшим качество модели, учитывая дисбаланс классов. Обучим разные модели и найдем лучшую;

4) Проведем финальное тестирование лучшей модели.

Приступим к исследованию! 

## Подготовка данных

### Импорт библиотек и знакомство с данными

In [36]:
#импортируем необходимые для проекта библиотеки и алгоритмы:
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from random import randint

In [8]:
#прочтем файл: 
df = pd.read_csv('/datasets/Churn.csv')
#выведем на экран первые 10 строк таблицы для ознакомления: 
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


### Предобработка данных

Для целей дальнейшего исследования **нам не понадобятся данные об индексе строки, уникальном идентификаторе клиента и фамилии**. Эти показатели не могу иметь какое-либо влияние на уход клиента из банка. Поэтому **удалим эти столбцы**. 

In [9]:
#удалим столбцы RowNumber, CustomerId, Surname: 
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)
#выведем на экран общую информацию о данных таблицы: 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Итак, в таблице 10000 строк, типы данных указаны верно, но **в столбце Tenure имеются пропуски - порядка 10% значений**. Посмотрим на строки с пропусками в этом столбце.

In [10]:
#выведем на экран первые 5 строк среза таблицы, где значения столбца Tenure отсутствуют: 
df[df['Tenure'].isna()].head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0


Сложно предположить, почему эти данные отсутствуют, так как здесь есть пользователи с различным количеством продуктов, активные клиенты и нет, с различным доходом и т.д. В данном случае заменять пропуски на какие-либо значения мы не можем - это может повлиять на обучение модели. Придется удалить строки с пропусками. 

In [11]:
#удалим строки в пропусками: 
df = df.dropna()
#для проверки еще раз выведем на экран общую информацию о данных таблицы: 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
CreditScore        9091 non-null int64
Geography          9091 non-null object
Gender             9091 non-null object
Age                9091 non-null int64
Tenure             9091 non-null float64
Balance            9091 non-null float64
NumOfProducts      9091 non-null int64
HasCrCard          9091 non-null int64
IsActiveMember     9091 non-null int64
EstimatedSalary    9091 non-null float64
Exited             9091 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


Отлично, данные почти готовы к исследованию. Осталось **масштабировать числовые признаки и преобразовать категориальные признаки в численные** методом прямого кодирования. 

### Масштабирование,  кодирование признаков, разбиение на выборки

In [12]:
#преобразуем категориальные признаки в численные
df = pd.get_dummies(df,drop_first=True)
#для проверки выведем на экран 5 строк таблицы: 
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Так как тестовой выборки у нас нет, то нам необходимо **исходные данные разбить на 3 группы**: 

*60% данных- обучающая выборка, 20% - валидационная, 20% - тестовая*. 

С помощью функции `train_test_split` мы сначала отделим 20 % для тестовой выборки,  затем от оставшихся данных отделим еще 25% для валидационной выборки, остальное- обучающая. 

In [13]:
#поделим массив данных на признаки (features_df) и целевой признак (target_df):
features_df = df.drop('Exited', axis=1)
target_df = df['Exited']
#выделим 20% данных для тестовой выборки(features_test, target_test):
features_without_test, features_test, target_without_test, target_test = train_test_split(
    features_df, target_df, test_size=0.2, random_state=1000000, shuffle=True, stratify=target_df)
#выделим 25% от оставшихся данных для валид-ной выборки(features_valid, target_valid),ост.- обучающая (features_train, target_train):
features_train, features_valid, target_train, target_valid = train_test_split(
    features_without_test, target_without_test, test_size=0.25, random_state=1000000, shuffle=True, stratify=target_without_test)
#для проверки выведем на экран размеры получившихся выборок методом shape: 
display('Размер features и target обучающей выборки:', features_train.shape, target_train.shape)
display('Размер features и target валидационной и тестовой выборок:', features_valid.shape, target_valid.shape, features_test.shape, target_test.shape)

'Размер features и target обучающей выборки:'

(5454, 11)

(5454,)

'Размер features и target валидационной и тестовой выборок:'

(1818, 11)

(1818,)

(1819, 11)

(1819,)

In [14]:
#проведем масштабирование числовых признаков: 
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
#дополнительно проведем масштабирование числовых признаков для выборки features_without_test- она понадобится нам при тестировании: 
scaler.fit(features_without_test[numeric])
features_without_test[numeric] = scaler.transform(features_without_test[numeric])
#для проверки выведем на экран 5 строк таблицы features_test: 
features_test.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
4539,0.700473,-0.170110,0.693273,-1.217846,0.811004,1,1,-0.782841,0,0,0
7365,-0.844792,-0.264850,1.733087,1.243123,-0.914120,1,1,-0.800463,0,1,0
4512,0.164781,1.156241,-1.039750,1.156804,-0.914120,0,0,0.983212,0,0,1
5800,-0.525437,-0.833286,1.386482,0.471449,-0.914120,1,1,0.361967,0,0,1
9396,0.515041,0.114108,-1.039750,-1.217846,0.811004,1,0,1.725335,0,1,0


Наши данные готовы к исследованию!

### Функция для расчета AUC-ROC и F1-меры

В ходе проекта нам не раз придется прибегнуть к расчету метрик качества- AUC-ROC и F1-меры. Для удобства заранее запишем функцию, которая на вход принимает параметры модели, обучающую и валидационную выборки, а на выходе дает метрики AUC-ROC и F1-мера.

In [15]:
#функция, которая на вход принимает параметры модели, на выходе дает метрики качества (AUC-ROC и F1-мера):
def quality_metrics (model, features_train, target_train,features_valid, target_valid):
    model.fit(features_train,target_train)
    predict_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predict_valid)#F-1-мера
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)#AUC-ROC
    return f1, auc_roc

**Вывод** 

На этапе подготовки данных мы импортировали необходимые для дальнейшей работы библиотеки и алгоритмы, ознакомились с общей информацией о данных в таблице, провели предобработку данных и получили 3 выборки: обучающая (features_train, target_train), тестовая (features_test, target_test) и валидационная (features_valid, target_valid), выделенные из исходного датасета в соотношении 3/1/1. 

Приступим к исследованию! 

## Исследование задачи

### Исследование баланса классов

Для начала посмотрим, как распределены ответы нашего целевого признака. 

In [16]:
#с помощью метода value_counts выведем на экран относительные частоты встречаемых ответов 0 и 1:
target_df.value_counts(normalize=True)

0    0.796062
1    0.203938
Name: Exited, dtype: float64

Данные несбалансированы. Значения "0" в 4 раза превышают "1". Посмотрим, какое влияние окажет несбалансированность классов на обучение моделей. 

Обучим 3 модели: модель решающего дерева, модель случайного леса и модель логистической регрессии с рандомными параметрами и без учета дисбаланса, проверим метрики качества: AUC-ROC и F1-мера.

### Обучение моделей без учета дисбаланса

**РЕШАЮЩЕЕ ДЕРЕВО**

In [17]:
#посчитаем метрики качества (AUC-ROC и F1-мера) для модели решающего дерева:
model_1 = DecisionTreeClassifier(random_state=1000000)
f1_1, auc_roc_1 = quality_metrics(model_1, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_1)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_1)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.5056039850560399

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.6944556727647313

**СЛУЧАЙНЫЙ ЛЕС**

In [18]:
#посчитаем метрики качества (AUC-ROC и F1-мера) для модели случайного леса:
model_2 = RandomForestClassifier(random_state=1000000)
f1_2, auc_roc_2 = quality_metrics(model_2, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_2)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_2)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


'F-1-мера модели решающего дерева на валидационной выборке:'

0.5601317957166392

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.8174389991747961

**ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ**

In [19]:
#посчитаем метрики качества (AUC-ROC и F1-мера) для модели логистической регрессии:
model_3 = LogisticRegression(random_state=1000000, solver='liblinear')
f1_3, auc_roc_3 = quality_metrics(model_3, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_3)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_3)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.3137254901960785

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.7520047984770051

**Вывод**

Представим результаты исследования качества моделей в виде таблицы:

In [20]:
results = pd.DataFrame({'модель' : ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'],
                       'F-1-мера' : [f1_1, f1_2, f1_3],
                       'AUC-ROC' : [auc_roc_1, auc_roc_2, auc_roc_3]})
results

,модель,F-1-мера,AUC-ROC
0,Решающее дерево,0.505604,0.694456
1,Случайный лес,0.560132,0.817439
2,Логистическая регрессия,0.313725,0.752005


В рамках исследования задачи был выявлен дисбаланс классов: значения "0" в 4 раза превышают "1".  Чтобы проверить, какое влияние окажет несбалансированность классов на обучение моделей, мы обучили 3 модели без учета дисбаланса. Результаты работы моделей модем наблюдать в таблице выше. 

При проверке качества моделей критерием AUC-ROC мы выявили значения выше 0,5, но важно помнить, что этот критерий по сравнению с F-1-мерой более устойчив к несбалансированным классам, а вот значения критерия F-1-меры далеки от идеала. В рамках проекта нам необходимо довести метрику F-1-мера до 0.59, на несбалансированных данных такого результата мы не получили. 

Улучшим качество модели, учитывая дисбаланс классов. 

## Борьба с дисбалансом

### Аргумент class_weight

В первую очередь, для борьбы с дисбалансом классов воспользуемся `аргументом class_weight` алгоритмов моделей решающего дерева, случайного леса и логистической регрессии. 

А также для улучшения качества моделей с помощью модуля GridSearchCV подберем оптимальные гиперпараметры. 

**РЕШАЮЩЕЕ ДЕРЕВО**

In [21]:
#с помощью модуля GridSearchCV подберем оптимальные гиперпараметры для модели решающего дерева (model_4):
model_4 = DecisionTreeClassifier(random_state=1000000, class_weight='balanced')
parametrs_4 = { 'max_depth': range (1,10),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid_4 = GridSearchCV(model_4, parametrs_4, cv=5, scoring='f1')
grid_4.fit(features_train,target_train)
display(grid_4.best_params_)

{'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 8}

In [22]:
#перезапишем модель(model_4) с оптимальными гиперпараметрами и найдем показатели качества AUC-ROC и F1-мера:
model_4 = DecisionTreeClassifier(random_state=1000000, max_depth=5, min_samples_leaf=3, min_samples_split=8, class_weight='balanced')
f1_4, auc_roc_4 = quality_metrics(model_4, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_4)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_4)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.542110358180058

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.8252244163498417

**СЛУЧАЙНЫЙ ЛЕС**

In [23]:
#с помощью модуля GridSearchCV подберем оптимальные гиперпараметры для модели случайного леса (model_5):
model_5 = RandomForestClassifier(random_state=1000000, class_weight='balanced')
parametrs_5 = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,11)}
grid_5 = GridSearchCV(model_5, parametrs_5, cv=5, scoring='f1')
grid_5.fit(features_train,target_train)
display(grid_5.best_params_)

{'max_depth': 9, 'n_estimators': 30}

In [24]:
#перезапишем модель(model_5) с оптимальными гиперпараметрами и найдем показатели качества AUC-ROC и F1-мера:
model_5 = RandomForestClassifier(random_state=1000000, n_estimators=30, max_depth=9, class_weight='balanced')
f1_5, auc_roc_5 = quality_metrics(model_5, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_5)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_5)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.592964824120603

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.8487026788392008

**ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ**

In [25]:
#обучим модель лог.регрессии(model_6) с учетом аргумента class_weight='balanced' и найдем показатели качества AUC-ROC и F1-мера:
model_6 = LogisticRegression(random_state=1000000, solver='liblinear', class_weight='balanced')
f1_6, auc_roc_6 = quality_metrics(model_6, features_train, target_train,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_6)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_6)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.47970479704797053

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.7582878974437306

**Представим результаты исследования качества моделей 
с учетом дисбаланса классов (с помощью аргумента class_weight) в виде таблицы**:

In [26]:
results = pd.DataFrame({'модель' : ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'],
                       'F-1-мера' : [f1_4, f1_5, f1_6],
                       'AUC-ROC' : [auc_roc_4, auc_roc_5, auc_roc_6]})
results

,модель,F-1-мера,AUC-ROC
0,Решающее дерево,0.542110,0.825224
1,Случайный лес,0.592965,0.848703
2,Логистическая регрессия,0.479705,0.758288


Благодаря учету баланса классов, а также подбору оптимальных гиперпараметров, улучшились метрики качества по всем моделям, а метрика F-1-мера модели случайного леса достигла необходимого уровня - 0,59. Пока можем считать эту модель лучшей. 

### Уменьшение выборки техникой downsampling

Для борьбы с дисбалансом классов попробуем еще один метод - технику downsampling. 

Создадим функцию, которая на выходе даст обучающую выборку, уменьшенную по технике downsampling. 

In [27]:
#запишем функцию
def downsampling (features, target, fraction):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=1000000)]+[features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=1000000)]+[target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=1000000)
    return features_downsampled, target_downsampled

In [28]:
#получим новую обучающую выборку: 
features_downsampled, target_downsampled = downsampling (features_train, target_train, 0.3)
#для проверки выведем на экран размеры получившихся выборок методом shape: 
display('Размер features_downsampled и target_downsampled :', features_downsampled.shape, target_downsampled.shape)

'Размер features_downsampled и target_downsampled :'

(2415, 11)

(2415,)

**Найдем показатели качества AUC-ROC и F1-мера только для модели случайного леса, так как ранее именно эта модель показала наилучший результат по метрике F-1-мера.**

Так как проводить кросс-валидацию на upsampled/downsampled данных – некорректно, так как баланс классов нарушен и  внутри кросс-валидации происходит разбиение переданной в нее выборки на треин и валидацию, **подберем параметры вручную**. 

**СЛУЧАЙНЫЙ ЛЕС**

In [39]:
#подберем оптимальные гиперпараметры для модели случайного леса (model_7) с учетом новой обучающей выборки:
best_model =  None
best_f1 = 0
best_est = 0
best_depth = 0
for est in range (10,41,10):
    for depth in range (1,10):
        model = RandomForestClassifier(random_state=1000000,n_estimators=est, max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predict_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predict_valid)
        if f1 > best_f1:
            best_model =  model
            best_f1 = f1
            best_est = est
            best_depth = depth
display('Оптимальные ГП для модели model_7 с наибольшим f1=',best_f1,': n_estimators=', best_est, ', max_depth=', best_depth)

'Оптимальные ГП для модели model_7 с наибольшим f1='

0.5910112359550562

': n_estimators='

20

', max_depth='

8

In [38]:
#найдем показатели качества AUC-ROC и F1-мера для случайного леса на новых обучающих данных:
model_7 = RandomForestClassifier(random_state=1000000, n_estimators=20, max_depth=8)
f1_7, auc_roc_7 = quality_metrics(model_7, features_downsampled, target_downsampled,features_valid, target_valid)
display('F-1-мера модели решающего дерева на валидационной выборке:', f1_7)
display('AUC-ROC модели решающего дерева на валидационной выборке:', auc_roc_7)

'F-1-мера модели решающего дерева на валидационной выборке:'

0.5910112359550562

'AUC-ROC модели решающего дерева на валидационной выборке:'

0.8434329228425016

Получены довольно высокие значения F1-меры и AUC-ROC, однако ниже, чем у модели случайного леса, обученной на сбалансированных данных с помощью аргумента class_weight.

Таким образом, модель, показавшая наилучший результат в рамках исследования - это model_5. 

**Вывод**

С целью улучшения качества моделей мы избавлялись от дисбаланса классов 2 методами: с помощью аргумента class_weight алгоритмов моделей, а также путем уменьшения обучающей выборки техникой downsampling. 

Наилучшие показатели качества выдала модель случайного леса, обученная на сбалансированных данных с помощью аргумента class_weight = 'balanced', с гиперпараметрами 'max_depth'= 9, 'n_estimators'= 30. Эта модель выдала следующие значения метрик качества на валидационных данных:  F-1-мера = 0.5929, AUC-ROC = 0.8486. Эту модель можем считать успешной, поскольку показатель F-1-меры=0.59, что удовлетворяет задаче нашего проекта. 

Протестируем эту модель на тестовой выборке. 

## Тестирование модели

Проверим нашу лучшую модель случайного леса `model_5` на тестовой выборке, найдем показатели качества AUC-ROC и F1-мера. 

In [31]:
#найдем показатели качества AUC-ROC и F1-мера на тестовой выборке: 
f1_test, auc_roc_test = quality_metrics(model_5, features_train, target_train,features_test, target_test)
display('F-1-мера модели решающего дерева на  тестовой выборке:', f1_test)
display('AUC-ROC модели решающего дерева на тестовой выборке:', auc_roc_test)

'F-1-мера модели решающего дерева на  тестовой выборке:'

0.6375321336760926

'AUC-ROC модели решающего дерева на тестовой выборке:'

0.8636282408303673

На тестовой выборке наша модель показала еще более высокие показатели качества: F-1-мера = 0.6375, AUC-ROC = 0.8636. 
Проверим, как изменятся эти показатели, если мы **проведем обучение модели на объединенных данных обучающей и валидационной выборок, а затем посчитаем метрики качества на тестовой выборке**.

In [32]:
#с помощью модуля GridSearchCV подберем оптимальные гиперпараметры для модели случайного леса (model_8), обученной на объединенных данных:
model_8 = RandomForestClassifier(random_state=1000000, class_weight='balanced')
parametrs_8 = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,11)}
grid_8 = GridSearchCV(model_8, parametrs_8, cv=5, scoring='f1')
grid_8.fit(features_without_test, target_without_test)
display(grid_8.best_params_)

{'max_depth': 9, 'n_estimators': 20}

In [40]:
#перезапишем модель(model_8) с оптимальными гиперпараметрами и найдем показатели качества AUC-ROC и F1-мера на тестовой выборке:
model_8 = RandomForestClassifier(random_state=1000000, n_estimators=20, max_depth=9, class_weight='balanced')
f1_8, auc_roc_8 = quality_metrics(model_8, features_without_test, target_without_test,features_test, target_test)
display('F-1-мера модели решающего дерева на тестовой выборке:', f1_8)
display('AUC-ROC модели решающего дерева на тестовой выборке:', auc_roc_8)

'F-1-мера модели решающего дерева на тестовой выборке:'

0.6191051995163241

'AUC-ROC модели решающего дерева на тестовой выборке:'

0.8636933925034623

После переобучения модели на объединенных данных качество осталось практически на том же уровне.

Для проверки модели на адекватность **проверим эффективность предсказаний модели, которая предсказывает рандомно 0 или 1**. 

In [34]:
#создадим список с рандомными предсказаниями 0 и 1 (random_predictions):
random_predictions = []
for _ in range(len(target_test)):
    random_predictions.append(randint(0,1))
#найдем значение F1-меры на фиктивных предсказаниях: 
f1_fict = f1_score(target_test, random_predictions)
#выведем на экран: 
display('F-1-мера модели решающего дерева на фиктивных предсказаниях:', f1_fict)

'F-1-мера модели решающего дерева на фиктивных предсказаниях:'

0.2992874109263658

Значение F-1-меры, равное 0.29, гораздо ниже значений любой исследованной в данном проекте модели. Таким образом, наша модель проверку на адекватность прошла. 

**Вывод**

В рамках тестирования модели мы проверили работу нашей лучшей модели на тестовой выборке. На тестовой выборке модель показала еще более высокие показатели качества, чем на валидационной: F-1-мера = 0.6392, AUC-ROC = 0.8635. 

Затем мы попробовали еще улучшить эти показатели, проведя обучение модели на объединенных данных обучающей и валидационной выборок, однако после переобучения модели на объединенных данных качество осталось практически на том же уровне.

И наконец, для проверки модели на адекватность мы сравнили результат метрики F-1-мера нашей модели с моделью, которая предсказывает рандомно 0 или 1. У фиктивной модели значение F-1-меры равно 0.29, что гораздо ниже значений любой исследованной в данном проекте модели. Таким образом, наша модель проверку на адекватность прошла.

## Общий вывод

Для подведения итогов исследования давайте еще раз вспомним *задачу, которая стояла перед нами в начале исследования*: 

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59.

*В рамках исследования* был выявлен дисбаланс классов: значения "0" в 4 раза превышают "1". Это и понятно, ситуация, когда из банка уходит половина клиентов была бы критичной. 

С целью улучшения качества моделей мы избавлялись от дисбаланса классов 2 методами: с помощью аргумента class_weight алгоритмов моделей, а также путем уменьшения обучающей выборки техникой downsampling.

Наилучшие показатели качества выдала модель случайного леса, обученная на сбалансированных данных с помощью аргумента class_weight = 'balanced', с гиперпараметрами 'max_depth'= 9, 'n_estimators'= 30. Эта модель выдала следующие значения метрик качества на валидационных данных: F-1-мера = 0.5929, AUC-ROC = 0.8487. Эту модель можем считать успешной, поскольку показатель F-1-меры=0.59, что удовлетворяет задаче нашего проекта.

На тестовой выборке модель показала еще более высокие показатели качества, чем на валидационной: F-1-мера = 0.6375, AUC-ROC = 0.8636.

Таким образом, *обученная модель удовлетворяет требованию заказчика (довести метрику до 0.59) и может быть использована в дальнейшем для предсказания ухода клиентов из банка и принятия мер по сохранению этих клиентов*. 